In [4]:
import seaborn as sns #library untuk memperindah visualisasi data
import matplotlib.pyplot as plt
import pandas as pd #library untuk mengolah data

#Setup Seaborn
sns.set_style(style='white')
sns.set_context(context='notebook', font_scale=1.3, rc={'figure.figsize':(16,9)})

In [5]:
dataku = pd.read_csv('lemma.csv') #Membaca File CSV
dataku.label.value_counts() #MEnghitung Jumlah record berdasarkan label

label
1    502
2    280
0    240
Name: count, dtype: int64

In [6]:
#proses training
import pickle #library menyimpan model
from sklearn.svm import SVC #library metode yang akan dipakai untuk membuat model
from sklearn.feature_extraction.text import TfidfVectorizer #library untuk merubah data text kedalam bentuk vektor
from sklearn.preprocessing import LabelEncoder #library untuk merubah data label string kedalam bentuk numerik

In [7]:
#Merubah data label
dictio_label ={
    0:"Netral",
    1:"Positif",
    2:"Negatif"
}

dataku["label"]=dataku['label'].map(dictio_label)
dataku.label

0       Negatif
1        Netral
2       Negatif
3        Netral
4       Positif
         ...   
1017    Negatif
1018    Negatif
1019     Netral
1020    Negatif
1021     Netral
Name: label, Length: 1022, dtype: object

In [8]:
dataku = dataku.dropna() #mendrop record yang terdapat data kosong
dataku.isnull().sum() #menghitung data kosong per kolom

x = dataku.lemma #mengambil data pada kolom lemma
y = dataku.label #mengambil data pada kolom label

In [9]:
encoder = LabelEncoder() #Memanggil class encoder
train_y = encoder.fit_transform(y) #encode data kolom label

tfidf_vect = TfidfVectorizer() #memanggil class untuk vektorizer

#Transform data pada kolom lemma kedalam bentuk vektor
tfidf_vect.fit(x) 
train_x_tfidf = tfidf_vect.transform(x)

SVM = SVC(decision_function_shape='ovo') #memanggil metode training
text_pickle =SVM.fit(train_x_tfidf, train_y) #membuat model

#save model
files = open('SVM_classifier_24.pickle','wb')
pickle.dump(text_pickle, files)
files.close()
print("selesai")

selesai


In [10]:
#Loading model yang sudah disimpan
model = open('SVM_classifier_24.pickle', 'rb') 
svm_classifier = pickle.load(model)
svm_classifier

SVC(decision_function_shape='ovo')

In [11]:
#Ekstrakasi fitur
from imblearn.over_sampling import SMOTE #Library untuk fitting data

vect = TfidfVectorizer(max_features=1000, binary=True) 
X = vect.fit_transform(dataku.lemma)

print(X)


#sm=SMOTE()

  (0, 246)	0.34784398131795813
  (0, 451)	0.4062029592835461
  (0, 496)	0.34784398131795813
  (0, 703)	0.34784398131795813
  (0, 358)	0.3104419984635892
  (0, 922)	0.36496095827980607
  (0, 8)	0.35578904981582404
  (0, 240)	0.06261927703426774
  (0, 852)	0.3345670225054814
  (1, 621)	0.3662296968227069
  (1, 725)	0.31983145432337434
  (1, 271)	0.27550031643960754
  (1, 596)	0.33113078132285434
  (1, 845)	0.31983145432337434
  (1, 704)	0.3456981474393126
  (1, 280)	0.31983145432337434
  (1, 97)	0.27990631426249085
  (1, 8)	0.3027934991351645
  (1, 240)	0.053291999898072415
  (1, 852)	0.2847325388235218
  (2, 337)	0.20159863811330936
  (2, 983)	0.48392982259139744
  (2, 990)	0.48392982259139744
  (2, 167)	0.43755070014896813
  (2, 284)	0.3762408351760998
  :	:
  (1010, 240)	0.08495333121022464
  (1011, 240)	1.0
  (1012, 898)	0.5727166512555294
  (1012, 195)	0.5979120532248099
  (1012, 149)	0.553173579195811
  (1012, 240)	0.09217269260925547
  (1013, 670)	0.8837026596111464
  (1013, 208)	

In [12]:
from sklearn.model_selection import ShuffleSplit #library untuk split data 
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix #library untuk evaluasi


X = dataku.lemma
Y = dataku.label

ss = ShuffleSplit(n_splits=10, test_size=0.2) # mensplit data
sm = SMOTE()

accs = []
fis = []
cms = []

for train_index, test_index in ss.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)

    X_train_res, y_train_res = sm.fit_resample(X_train_vect, y_train) #FITTING DATA
    
    SVM.fit(X_train_res, y_train_res)
    y_pred = SVM.predict(X_test_vect)#memprediksi 

    accs.append(accuracy_score(y_test,y_pred)) #menyimpan akurasi
    fis.append(f1_score(y_test, y_pred, average='weighted')) #menyimpan nilai f-1
    cms.append(confusion_matrix(y_test,y_pred)) #menyimpan confusion matrix

print(classification_report(y_test,y_pred))
print("\naverage accuracy across fold : {:2f}%".format(sum(accs)/len(accs) *100))
print("\naverage F1-score across fold : {:2f}%".format(sum(fis)/len(fis) *100))
print("\naverage Confusion Matrix across fold : \n {}".format(sum(cms)/len(cms)))

              precision    recall  f1-score   support

     Negatif       0.56      0.80      0.66        51
      Netral       0.69      0.54      0.60        41
     Positif       0.91      0.80      0.85       112

    accuracy                           0.75       204
   macro avg       0.72      0.71      0.71       204
weighted avg       0.78      0.75      0.75       204


average accuracy across fold : 71.470588%

average F1-score across fold : 71.200694%

average Confusion Matrix across fold : 
 [[36.1  8.9 10.1]
 [13.8 24.7  8.5]
 [10.8  6.1 85. ]]


In [13]:
model = open('SVM_classifier_24.pickle','rb')
svm_classifier = pickle.load(model)
model.close

<function BufferedReader.close>

In [22]:
input_text = ""
text_vector = tfidf_vect.transform([input_text])
predicted_label = encoder.inverse_transform(svm_classifier.predict(text_vector))
print(f"prediksi sentimen teks: {predicted_label[0]}")

prediksi sentimen teks: Negatif
